In [1]:
import pandas as pd
import numpy as np

In [2]:

#Load in raw excel data
EXP = pd.read_excel(r'MasterNuclei.xlsx', sheet_name = 'Exp') #experimental


ME2 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'DD-ME2_even-even_nuclei')
MEdelta = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'DD-MEdelta_even-even_nuclei')
PC1 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'DD-PC1_even-even_nuclei')
NL3S = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'NL3S_even-even_nuclei')


SKMS = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'SKMS_all_nuclei')
SKP = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'SKP_all_nuclei')
SLY4 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'SLY4_all_nuclei')
SV = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'SV-MIN_all_nuclei')
UNEDF0 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'UNEDF0_all_nuclei')
UNEDF1 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'UNEDF1_all_nuclei')

SKMS_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'SKMS_even-even_nuclei')
SKP_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'SKP_even-even_nuclei')
SLY4_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'SLY4_even-even_nuclei')
SV_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'SV-MIN_even-even_nuclei')
UNEDF0_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'UNEDF0_even-even_nuclei')
UNEDF1_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'UNEDF1_even-even_nuclei')


#raw = tuple([EXP, ME2, MEdelta, PC1, NL3S, SKMS, SKP, SLY4, SV, UNEDF0, UNEDF1])

In [3]:

def BE(Z,N,W,df):
    try:
        return float(df[(df['Z']==Z) & (df['N']==N) & (df['Wigner']==W)]['BE'])
    except:
        return None

def OneNSE(Z,N,W,df):
    try:
        return BE(Z,N,W,df) - BE(Z,N-1,W,df)
    except:
        return None
    
def OnePSE(Z,N,W,df):
    try:
        return BE(Z,N,W,df) - BE(Z-1,N,W,df)
    except:
        return None

def TwoNSE(Z,N,W,df):
    try:
        return BE(Z,N,W,df) - BE(Z,N-2,W,df)
    except:
        return None

def TwoPSE(Z,N,W,df):
    try:
        return BE(Z,N,W,df) - BE(Z-2,N,W,df)
    except:
        return None

def AlphaSE(Z,N,W,df):
    try:
        return BE(Z,N,W,df) - BE(Z-2,N-2,W,df) - 28.3
    except:
        return None

def TwoPSGap(Z,N,W,df):
    try:
        return TwoPSE(Z,N,W,df) - TwoPSE(Z+2,N,W,df)
    except:
        return None

def TwoNSGap(Z,N,W,df):
    try:
        return TwoNSE(Z,N,W,df) - TwoNSE(Z,N+2,W,df)
    except:
        return None

def DoubleMDiff(Z,N,W,df):
    try:
        return (TwoPSE(Z,N,W,df) - TwoPSE(Z,N-2,W,df))/4
    except:
        return None

def N3PointOED(Z,N,W,df):
    try:
        return (-1)**N*(OneNSE(Z,N,W,df) - OneNSE(Z,N+1,W,df))/2
    except:
        return None

def P3PointOED(Z,N,W,df):
    try:
        return (-1)**Z*(OnePSE(Z,N,W,df) - OnePSE(Z+1,N,W,df))/2
    except:
        return None

def SNESplitting(Z,N,W,df):
    try:
        return (-1)**N*(OneNSE(Z,N,W,df) - OneNSE(Z,N+2,W,df))
    except:
        return None

def SPESplitting(Z,N,W,df):
    try:
        return (-1)**Z*(OnePSE(Z,N,W,df) - OnePSE(Z+2,N,W,df))
    except:
        return None

def WignerEC(Z,N,W,df):
    if Z!=N:
        return None
    try:
        return DoubleMDiff(Z,N,W,df)-(DoubleMDiff(Z-2,N,W,df) + DoubleMDiff(Z,N+2,W,df))/2
    except:
        return None
    
def BEperA(Z,N,W,df):
    try:
        return BE(Z,N,W,df)/(Z+N)
    except:
        return None

In [4]:
def Wig1(Z, N):
    return (1.8*np.exp(-380*((N-Z)/(N+Z))**2))-(.84*abs(N-Z)*np.exp(-(((N+Z)/26)**2)))

def Wig2(Z, N):
    return -47*(abs(N-Z)/(N+Z))

In [5]:
#Collect all raw model data into each individual df

covar = [ME2, MEdelta, PC1, NL3S]

skm = [SKMS, SKP, SLY4, SV, UNEDF0, UNEDF1]
skm_even = [SKMS_even, SKP_even, SLY4_even, SV_even, UNEDF0_even, UNEDF1_even]
skm_merged = []
for i in range(len(skm)):
    skm_merged.append(skm[i].combine_first(skm_even[i]))

raw = [EXP]

for c in covar:
    raw.append(c)
for s in skm_merged:
    raw.append(s)

In [6]:
filtered = []
for d in raw:
    df = d.loc[:,['Z', 'N', 'Binding_Energy_(MeV)']]
    df['Binding_Energy_(MeV)'] = abs(df['Binding_Energy_(MeV)'])
    filtered.append(df)
filtered[3]

,Z,N,Binding_Energy_(MeV)
0,2,2,21.017
1,2,4,27.777
2,2,6,30.309
3,4,2,24.499
4,4,4,46.425
...,...,...,...
1756,116,176,2077.402
1757,116,178,2090.569
1758,118,174,2067.141
1759,118,176,2080.712


In [7]:
def Identity(Z, N):
    return 0
wigs = [Identity, Wig1, Wig2]
wigner_dfs =[]
for df in filtered:
    new_df = pd.DataFrame(columns=['Z', 'N', 'Wigner', 'BE'])
    for wig in range(3):
        for rowi in range(len(df.index)):
            newBE = df.loc[rowi, 'Binding_Energy_(MeV)'] - wigs[wig](df.iloc[rowi,0], df.iloc[rowi,1])
            new_df.loc[len(new_df.index)] = [df.loc[rowi,'Z'], df.loc[rowi,'N'], wig, newBE]
    wigner_dfs.append(new_df)

In [8]:
mnames = ['EXP', 'ME2', 'MEdelta', 'PC1', 'NL3S', 'SKMS', 'SKP', 'SLY4', 'SV', 'UNEDF0', 'UNEDF1']
qfuncs = [OneNSE, OnePSE, TwoNSE, TwoPSE, AlphaSE, TwoNSGap, TwoPSGap, DoubleMDiff, N3PointOED, P3PointOED, SNESplitting, SPESplitting, WignerEC, BEperA]
qnames = ['OneNSE', 'OnePSE', 'TwoNSE', 'TwoPSE', 'AlphaSE', 'TwoNSGap', 'TwoPSGap', 'DoubleMDiff', 'N3PointOED', 'P3PointOED', 'SNESplitting', 'SPESplitting', 'WignerEC', 'BE/A']
# quan_df = pd.DataFrame(
#     columns=['One Neutron Separation Energy', 'One Proton Separation Energy', 'Two Neutron Separation Energy', 
#     'Two Proton Separation Energy', 'Alpha Separation Energy', 'Two Proton Shell Gap', 'Two Neutron Shell Gap', 
#     'Double Mass Difference', 'Neutron 3-Point Odd-Even Binding Energy Difference', 'Proton 3-Point Odd-Even Binding Energy Difference',
#     'Single-Neutron Energy Splitting', 'Single-Proton Energy Splitting', 'Wigner Energy Coeffienct', 'Binding Energy/Nucleon']
# )

quan_dfs = []
for model in wigner_dfs:
    print('checkpoint')
    df = pd.DataFrame(columns=qnames)
    for rowi in range(len(model.index)):
        df.loc[len(df.index)] = [q(model.loc[rowi, 'Z'], model.loc[rowi, 'N'], model.loc[rowi, 'Wigner'], model) for q in qfuncs]
    quan_dfs.append(df)

checkpoint
checkpoint
checkpoint
checkpoint
checkpoint
checkpoint
checkpoint
checkpoint
checkpoint
checkpoint
checkpoint


In [9]:
final_dfs = [pd.concat([wigner_dfs[i], quan_dfs[i]], axis=1) for i in range(len(wigner_dfs))]

In [10]:
final_dfs[0]

,Z,N,Wigner,BE,OneNSE,OnePSE,TwoNSE,TwoPSE,AlphaSE,TwoNSGap,TwoPSGap,DoubleMDiff,N3PointOED,P3PointOED,SNESplitting,SPESplitting,WignerEC,BE/A
0,0.0,1.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,1.0,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2,1.0,1.0,0.0,2.224566,2.224566,2.224566,NaN,NaN,NaN,NaN,NaN,NaN,2.016332,1.634454,-3.824566,-5.327566,NaN,1.112283
3,1.0,2.0,0.0,8.481796,6.257230,NaN,8.481796,NaN,NaN,10.281796,NaN,NaN,3.928615,NaN,6.457230,NaN,NaN,2.827265
4,2.0,1.0,0.0,7.718041,NaN,5.493475,NaN,7.718041,NaN,NaN,NaN,NaN,NaN,4.298238,NaN,NaN,NaN,2.572680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7645,108.0,157.0,2.0,1942.196127,6.879355,NaN,NaN,NaN,-10.607859,NaN,NaN,NaN,0.548278,NaN,NaN,NaN,NaN,7.329042
7646,108.0,158.0,2.0,1950.172039,7.975912,NaN,14.855268,NaN,-10.484909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.331474
7647,109.0,157.0,2.0,1942.503310,NaN,0.307183,NaN,NaN,-11.129449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.302644
7648,110.0,159.0,2.0,1958.853060,NaN,NaN,NaN,NaN,-11.643067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.281982


In [13]:
for i in range(len(final_dfs)):
    final_dfs[i].to_hdf('May23.h5', mnames[i])